In [1]:
# 导入包文件
from tqdm import tqdm
import numpy as np
import time
import datetime# 获取数据
from download_data_export_thailand import downloadexport
import download_data_export_thailand


In [2]:
# 下载数据
url = "https://dataapi.moc.go.th/export-harmonize-countries"
a = downloadexport(url=url,hs_code='081060',star_year=2020,star_month=1,end_year=2021,end_month=10)
a.to_excel()
data = a.down_data()

In [ ]:
# index转换为datetime格式
date = [datetime.datetime.strptime(i,'%Y%M') for i in data.index.values]
date = [i.strftime('%YY%mM') for i in date]
data['date'] = date
data.set_index(keys = 'date',inplace = True)
data.head()

In [ ]:
# 数据分类
data_all = data.groupby(data.index).sum()
data_china = data[data['country_name_en'] == 'CHINA']
data_hk = data[data['country_name_en'] == 'HONG KONG']
data_else_country = data[~(data['country_name_en'].str.contains('CHINA') | data['country_name_en'].str.contains('HONG KONG'))]
# 将国外数据分类
data_else_country = data_else_country.groupby(data_else_country.index).sum()
a = list(map(lambda x:round(x/17280,0),data_all.quantity.values.tolist()))
a

In [23]:
# 绘图
import pyecharts.options as opts
from pyecharts.charts import Line

c = (
    Line()
    # 
    
    .add_xaxis(data_china.index.values.tolist())
    .add_yaxis(
        "ALL",
        list(map(lambda x:round(x/17280,0),data_all.quantity.values.tolist())),
        # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),

    )
    .add_yaxis(
        "CHINA",
        list(map(lambda x:round(x/17280,0),data_china[data_china.country_name_en == 'CHINA'].quantity.values.tolist())),
        # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),

    )
    .add_yaxis(
        "HONG KONG",
        list(map(lambda x:round(x/17280,0),data_hk[data_hk.country_name_en == 'HONG KONG'].quantity.values.tolist())),
        # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
    )
    .add_yaxis(
        "ELSE COUNTRY",
        list(map(lambda x:round(x/17280,0),data_else_country.quantity.values.tolist())),
        # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="2020-2021年泰国榴莲出口数据（柜/月）"))
    .render(r"line_markline.html")
)
